Imports

In [1]:
from collections import Counter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from BiLstmClassifier import BiLstmFixedLength, BiLstmVariableLength, BiLstmGloveVector

import numpy as np
import pandas as pd
import spacy
import torch
import torch.nn.functional as F

## 0. Hyper Parameters Definition
Word embeddings are always around 50 and 300 in length, longer embedding vectors don't add enough information and
smaller ones don't represent the semantics well enough.


In [2]:
EPOCHS = 100
BATCH_SIZE = 100  # Small batches because the dataset is not bigger than 500 rows
HIDDEN_LAYER_DIM = 60  # AOLME is not too complex language, it represents the language's features
EMBEDDED_LAYER_DIM = 50

## 1. Load dataset

In [3]:
roles = pd.read_csv('output/balanced_20201008235520.csv')
print(roles.shape)
print(roles.head())

(372, 2)
      Role                           Text
0  Student          you like how its like
1  Student  its like youre obsessed with 
2  Student                   and this one
3  Student                             no
4  Student           i dont like summary 


## 2. Mapping 'Roles' labels to numbers for vectorization

In [4]:
mapping = {'Student': 0, 'Co-Facilitator': 1, 'Facilitator': 2}
roles['Role'] = roles['Role'].apply(lambda x: mapping[x])
roles.head()


# Load English words model package
tok = spacy.load('en')


def tokenize(text: str):
    """
    This method tokenizes a sentence, considering the text is already lowered,
    ASCII, and  punctuation has been removed
    :param text: The sentence to be tokenized
    :return: A list containing each word of the sentence
    """
    return [token.text for token in tok.tokenizer(text)]


## 3. Dataset cleaning

In [5]:
# Count number of occurrences of each word
counts = Counter()
for index, row in roles.iterrows():
    counts.update(tokenize(row['Text']))

# Deletes words appearing only once
print("num_words before:", len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:", len(counts.keys()))

# Creates vocabulary
vocab2index = {"": 0, "UNK": 1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)


def encode_sentence(text, vocabulary_map, n=70):
    """
    Encodes the sentence into a numerical vector, based on the vocabulary map
    :param text: The sentence
    :param vocabulary_map: A map assigning a number to each word in the vocabulary
    :param n: Required vector size
    :return: Vectorized sentence and length
    """
    tokenized = tokenize(text)
    vectorized = np.zeros(n, dtype=int)
    enc1 = np.array([vocabulary_map.get(w, vocabulary_map["UNK"]) for w in tokenized])
    length = min(n, len(enc1))
    vectorized[:length] = enc1[:length]
    return vectorized, length


# Creates a new column into Dataset: each sentence expressed as a numeric vector
roles['Vectorized'] = roles['Text'].apply(lambda x: np.array(encode_sentence(x, vocab2index)))
print(roles.head())

num_words before: 662
num_words after: 347
   Role                           Text  \
0     0          you like how its like   
1     0  its like youre obsessed with    
2     0                   and this one   
3     0                             no   
4     0           i dont like summary    

                                          Vectorized  
0  [[2, 3, 4, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
1  [[5, 3, 2, 1, 1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
2  [[7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
3  [[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
4  [[11, 12, 13, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  


<ipython-input-5-81d6e3e92ab5>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  roles['Vectorized'] = roles['Text'].apply(lambda x: np.array(encode_sentence(x, vocab2index)))


### Check if the dataset is balanced

In [6]:
Counter(roles['Role'])

Counter({0: 124, 1: 124, 2: 124})

### Split into training and validation partitions


In [7]:
X = list(roles['Vectorized'])
y = list(roles['Role'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)


class RolesDataset(Dataset):
    """
    Simple PyTorch Dataset wrapper defined by an array of vectorized sentences (X) and the role for each sentence (y)
    """
    def __init__(self, input_x, input_y):
        self.X = input_x
        self.y = input_y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]


training_ds = RolesDataset(X_train, y_train)
validation_ds = RolesDataset(X_valid, y_valid)

Functions

In [8]:
def train_model(input_model, epochs=10, lr=0.001):
    """
    Trains the input model
    :param input_model: Input Model
    :param epochs: The number of training epochs
    :param lr: Learning Rate
    """
    parameters = filter(lambda p: p.requires_grad, input_model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)

    for i in range(epochs):
        input_model.train()
        sum_loss = 0.0
        total = 0

        # Iterates on Training DataLoader
        for x, y, l in training_dl:
            x = x.long()
            y = y.long()
            y_pred = input_model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item() * y.shape[0]
            total += y.shape[0]

        val_loss, val_acc, val_rmse = get_metrics(input_model, validation_dl)

        if i % 10 == 1:
            print(f"Epoch {i}: training loss %.3f, valid. loss %.3f, valid. accuracy %.3f, and valid. RMSE %.3f" % (
                sum_loss / total, val_loss, val_acc, val_rmse))

    print(f"FINAL: training loss %.3f, valid. loss %.3f, valid. accuracy %.3f, and valid. RMSE %.3f" % (
        sum_loss / total, val_loss, val_acc, val_rmse))


def get_metrics(input_model, valid_dl):
    """
    Obtains current validation metrics
    :param input_model: Input Model
    :param valid_dl: Validation PyTorch DataLoader
    :return:
    """
    input_model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0

    # PyTorch uses CrossEntropy function to implement Softmax on the same function
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = input_model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item() * y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1))) * y.shape[0]
    return sum_loss / total, correct / total, sum_rmse / total

In [9]:
vocab_size = len(words)
training_dl = DataLoader(training_ds, batch_size=BATCH_SIZE, shuffle=True)
validation_dl = DataLoader(validation_ds, batch_size=BATCH_SIZE)

## BiLSTM - Fixed Length


In [10]:
model_fixed = BiLstmFixedLength(vocab_size, EMBEDDED_LAYER_DIM, HIDDEN_LAYER_DIM)

print(f'\nBiLSTM - Fixed Length: {EPOCHS} epochs, Learning Rate: 0.1')
print('=============================================================')
train_model(model_fixed, epochs=EPOCHS, lr=0.1)
print(f'\nBiLSTM - Fixed Length: {EPOCHS} epochs, Learning Rate: 0.05')
print('=============================================================')
train_model(model_fixed, epochs=EPOCHS, lr=0.05)
print(f'\nBiLSTM - Fixed Length: {EPOCHS} epochs, Learning Rate: 0.01')
print('=============================================================')
train_model(model_fixed, epochs=EPOCHS, lr=0.01)


BiLSTM - Fixed Length: 100 epochs, Learning Rate: 0.1
Epoch 1: training loss 1.319, valid. loss 1.081, valid. accuracy 0.387, and valid. RMSE 1.222
Epoch 11: training loss 0.353, valid. loss 1.387, valid. accuracy 0.573, and valid. RMSE 0.931
Epoch 21: training loss 0.231, valid. loss 1.984, valid. accuracy 0.533, and valid. RMSE 1.013
Epoch 31: training loss 0.201, valid. loss 2.211, valid. accuracy 0.507, and valid. RMSE 1.007
Epoch 41: training loss 0.603, valid. loss 2.523, valid. accuracy 0.493, and valid. RMSE 1.052
Epoch 51: training loss 0.388, valid. loss 2.242, valid. accuracy 0.467, and valid. RMSE 1.026
Epoch 61: training loss 0.272, valid. loss 2.490, valid. accuracy 0.467, and valid. RMSE 0.966
Epoch 71: training loss 0.315, valid. loss 2.470, valid. accuracy 0.520, and valid. RMSE 0.980
Epoch 81: training loss 0.289, valid. loss 2.418, valid. accuracy 0.547, and valid. RMSE 0.945
Epoch 91: training loss 0.228, valid. loss 2.091, valid. accuracy 0.533, and valid. RMSE 0.

## BiLSTM - Variable Length

In [11]:
model = BiLstmVariableLength(vocab_size, EMBEDDED_LAYER_DIM, HIDDEN_LAYER_DIM)

print(f'\nBiLSTM - Variable Length: {EPOCHS} epochs, Learning Rate: 0.1')
print('=============================================================')
train_model(model, epochs=EPOCHS, lr=0.1)
print(f'\nBiLSTM - Variable Length: {EPOCHS} epochs, Learning Rate: 0.05')
print('=============================================================')
train_model(model, epochs=EPOCHS, lr=0.05)
print(f'\nBiLSTM - Variable Length: {EPOCHS} epochs, Learning Rate: 0.01')
print('=============================================================')
train_model(model, epochs=EPOCHS, lr=0.01)


BiLSTM - Variable Length: 100 epochs, Learning Rate: 0.1
Epoch 1: training loss 1.100, valid. loss 1.238, valid. accuracy 0.347, and valid. RMSE 0.980
Epoch 11: training loss 0.344, valid. loss 1.634, valid. accuracy 0.520, and valid. RMSE 1.020
Epoch 21: training loss 0.275, valid. loss 1.796, valid. accuracy 0.453, and valid. RMSE 1.033
Epoch 31: training loss 0.287, valid. loss 1.873, valid. accuracy 0.600, and valid. RMSE 0.872
Epoch 41: training loss 0.342, valid. loss 2.175, valid. accuracy 0.613, and valid. RMSE 0.864
Epoch 51: training loss 0.340, valid. loss 2.042, valid. accuracy 0.573, and valid. RMSE 0.993
Epoch 61: training loss 0.377, valid. loss 2.014, valid. accuracy 0.547, and valid. RMSE 1.046
Epoch 71: training loss 0.403, valid. loss 2.018, valid. accuracy 0.613, and valid. RMSE 0.952
Epoch 81: training loss 0.336, valid. loss 2.092, valid. accuracy 0.507, and valid. RMSE 1.065
Epoch 91: training loss 0.381, valid. loss 1.819, valid. accuracy 0.547, and valid. RMSE

## BiLSTM - with pretrained GloVe Word Embeddings

In [12]:
def load_glove_vectors():
    """Load the glove Global Vectors for Word Representation"""
    word_vectors = {}

    with open("./data/glove/glove.6B.50d.txt", encoding="utf8") as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors


def get_embedding_matrix(word_counts, emb_size=50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32')  # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)  # adding a vector for unknown words
    vocab_to_idx["UNK"] = 1
    i = 2

    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25, 0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1
    return W, np.array(vocab), vocab_to_idx


word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_embedding_matrix(counts, EMBEDDED_LAYER_DIM)


model = BiLstmGloveVector(vocab_size, EMBEDDED_LAYER_DIM, HIDDEN_LAYER_DIM, pretrained_weights)

print(f'\nBiLSTM - with pretrained GloVe Word Embeddings: {EPOCHS} epochs, Learning Rate: 0.1')
print('====================================================================================')
train_model(model, epochs=EPOCHS, lr=0.1)
print(f'\nBiLSTM - with pretrained GloVe Word Embeddings: {EPOCHS} epochs, Learning Rate: 0.05')
print('====================================================================================')
train_model(model, epochs=EPOCHS, lr=0.05)
print(f'\nBiLSTM - with pretrained GloVe Word Embeddings: {EPOCHS} epochs, Learning Rate: 0.01')
print('====================================================================================')
train_model(model, epochs=EPOCHS, lr=0.01)




BiLSTM - with pretrained GloVe Word Embeddings: 100 epochs, Learning Rate: 0.1
Epoch 1: training loss 1.438, valid. loss 1.236, valid. accuracy 0.320, and valid. RMSE 0.825
Epoch 11: training loss 0.930, valid. loss 1.024, valid. accuracy 0.427, and valid. RMSE 1.026
Epoch 21: training loss 0.701, valid. loss 1.352, valid. accuracy 0.440, and valid. RMSE 1.020
Epoch 31: training loss 0.671, valid. loss 1.310, valid. accuracy 0.480, and valid. RMSE 1.020
Epoch 41: training loss 0.609, valid. loss 1.275, valid. accuracy 0.427, and valid. RMSE 1.102
Epoch 51: training loss 0.534, valid. loss 1.475, valid. accuracy 0.480, and valid. RMSE 0.980
Epoch 61: training loss 0.492, valid. loss 1.532, valid. accuracy 0.493, and valid. RMSE 0.993
Epoch 71: training loss 0.412, valid. loss 1.508, valid. accuracy 0.467, and valid. RMSE 1.102
Epoch 81: training loss 0.465, valid. loss 1.669, valid. accuracy 0.480, and valid. RMSE 0.980
Epoch 91: training loss 0.372, valid. loss 1.899, valid. accuracy 